- exp005&exp006

In [1]:
import pandas as pd
from pathlib import Path

class CFG:
    name = "ens001"
    seed = 42
    n_fold = 5

    path_input = Path("../input")
    path_output = Path("../output")

In [2]:
def setup(CFG):
    # expフォルダを作成
    CFG.path_exp = CFG.path_output / CFG.name
    CFG.path_exp.mkdir(parents=True, exist_ok=True)

    return CFG

CFG = setup(CFG)

In [3]:
sub_exp005 = pd.read_csv("../output/exp015/submission.csv")
sub_exp006 = pd.read_csv("../output/exp016/submission.csv")

In [4]:
sub_exp005['predict'] = sub_exp005.apply(lambda row: ' '.join(row.astype(str)), axis=1)
sub_exp006['predict'] = sub_exp006.apply(lambda row: ' '.join(row.astype(str)), axis=1)

In [5]:
def cust_blend(dt, W = [1,1,1]):
    REC = []
    REC.append(dt['predict_0'].split())
    REC.append(dt['predict_1'].split())

    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 10 itens only
    return ' '.join(res[:10])

In [6]:
sub = sub_exp005[['predict']]
sub['predict1'] = sub_exp006['predict']
sub.columns = [f'predict_{i}' for i in range(len(sub.columns))]

/var/folders/3l/2bkf0scn43g79fkq6f3ft9z40000gn/T/ipykernel_92114/3224590921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['predict1'] = sub_exp006['predict']


In [7]:
sub

,predict_0,predict_1
0,3560 4420 9830 4714 2911 9534 13192 5466 2680 ...,3560 2986 4714 9830 11561 2680 5466 2305 4420 ...
1,5627 2439 9323 11923 12350 10095 12862 6555 79...,2439 9323 11923 12862 613 11237 8108 6555 7014...
2,757 9190 9910 410 6721 10485 7710 1774 6730 3400,757 9190 410 9910 1774 10485 7710 6721 6730 3400
3,12341 1542 10746 13521 3359 5080 6991 9319 565...,12341 13610 3359 6991 5657 7049 277 9319 4180 ...
4,607 763 11480 5650 6576 1448 3476 6161 3844 12029,607 763 11480 5650 1448 7246 12524 3187 6576 5106
...,...,...
174695,1997 7888 11123 2278 5744 7062 9543 10997 1184...,1997 11123 2278 7888 5744 7062 9543 10997 9743...
174696,6874 2164 4014 1227 13702 3644 5331 2232 13220...,6874 2232 1227 2164 9113 13702 4014 9723 3644 ...
174697,11037 12240 2087 13797 7379 7308 844 11796 814...,11037 2087 12240 7379 7308 13797 5810 13719 81...
174698,13672 4483 1687 6034 2510 3002 2373 5513 2305 ...,13672 1687 4483 5515 2407 11496 2373 3002 1228...


In [8]:
# 今回はweightを1:1とする。
sub['predict'] = sub.apply(cust_blend, W = [2.00,1.00], axis=1)

/var/folders/3l/2bkf0scn43g79fkq6f3ft9z40000gn/T/ipykernel_92114/1513419769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['predict'] = sub.apply(cust_blend, W = [2.00,1.00], axis=1)


In [9]:
ensemble_sub = sub['predict'].str.split(' ',expand=True)

In [10]:
ensemble_sub.columns = [f'predict_{i}' for i in range(ensemble_sub.shape[1])]


In [11]:
ensemble_sub

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,3560,4420,9830,4714,2986,2911,11561,5466,2680,9534
1,5627,2439,9323,11923,12862,12350,6555,10095,7913,6129
2,757,9190,9910,410,6721,10485,1774,7710,6730,3400
3,12341,1542,10746,3359,6991,13521,13610,5657,9319,5080
4,607,763,11480,5650,1448,6576,3476,6161,3844,12029
...,...,...,...,...,...,...,...,...,...,...
174695,1997,7888,11123,2278,5744,7062,9543,10997,11848,9743
174696,6874,2164,1227,4014,2232,13702,3644,13220,5331,3802
174697,11037,12240,2087,13797,7379,7308,8143,12939,844,11796
174698,13672,4483,1687,6034,3002,2373,2510,5513,5515,2305


In [12]:
ensemble_sub.to_csv(CFG.path_exp / "submission.csv", index=False)